> Proszę zaimplementować algorytm sortowania listy jednokierunkowej. W szczególności należy:

1. Zdefiniować klasę w Pythonie realizującą listę jednokierunkową.

2. Zaimplementować wstawianie do posortowanej listy.

3. Zaimplementować usuwanie maksimum z listy.

4. Zaimplementować sortowanie przez wstawianie lub sortowanie przez wybieranie na podstawie powyższych funkcji

### #1 Deklaracja klasy, reprezentującej listę jednokierunkową

In [1]:
class Node:
    def __init__(self, val):
        self.next = None
        self.val = val
        
        
class LinkedList:
    def __init__(self, values=None):
        self.head = self.tail = None
        self.length = 0
        values and self.extend(values)
        
    def __len__(self):
        return self.length
    
    def __iter__(self):
        curr = self.head
        while curr:
            yield curr.val
            curr = curr.next
    
    def __str__(self):
        return ' -> '.join(str(v) for v in self)
        
    def append(self, val):
        node = Node(val)
        if not self:
            self.head = self.tail = node
        else:
            self.tail.next = node
            self.tail = node
        self.length += 1
        
    def prepend(self, val):
        node = Node(val)
        if not self:
            self.head = self.tail = node
        else:
            node.next = self.head
            self.head = node
        self.length += 1
        
    def extend(self, values):
        for val in values:
            self.append(val)
            
    def popleft(self):
        if not self:
            raise IndexError(f'pop from an empty {self.__class__.__name__}')
        removed = self.head.val
        self.head = self.head.next
        self.length -= 1
        return removed
            
    def insert_after(self, val, node):
        new_node = Node(val)
        new_node.next = node.next
        node.next = new_node
        self.length += 1
        
    def remove_next_node(self, node):
        if node is not self.tail:
            removed = node.next
            if removed is self.tail:
                self.tail = node
            node.next = removed.next
            self.length -= 1
            return removed

###### Przykładowy test

In [2]:
import random

sorted_lst = sorted([random.randint(-100, 100) for _ in range(6)])
ll = LinkedList(sorted_lst)

print(ll)

-68 -> -65 -> -20 -> 5 -> 44 -> 83


### #2 Wstawianie do posortowanej listy

In [3]:
def insert_val(ll, val):  # Linked List ll must be sorted in an increasing order
    if not ll or val > ll.tail.val:
        ll.append(val)
    elif val < ll.head.val:
        ll.prepend(val)
    else:
        curr = ll.head
        while val > curr.next.val:
            curr = curr.next
        ll.insert_after(val, curr)

###### Przykładowy test

In [4]:
ll = LinkedList(sorted_lst)

print(ll)
insert_val(ll, 50)
insert_val(ll, -100)
insert_val(ll, 100)
insert_val(ll, 0)
print(ll)

-68 -> -65 -> -20 -> 5 -> 44 -> 83
-100 -> -68 -> -65 -> -20 -> 0 -> 5 -> 44 -> 50 -> 83 -> 100


### #3 Usuwanie maksimum z listy

In [5]:
def remove_max(ll):
    # Add a sentinel node
    ll.prepend(None)
    # Create pointers to previous nodes
    max_prev = ll.head
    curr_prev = ll.head.next
    # Loop till the current node exists
    while curr_prev.next:
        if curr_prev.next.val > max_prev.next.val:
            max_prev = curr_prev
        curr_prev = curr_prev.next
        
    # Remove a maximum value from a Linked List
    removed = ll.remove_next_node(max_prev)
    # Remove a sentinel node
    ll.popleft()
    
    return removed.val

###### Przykładowy test

In [6]:
values = [random.randint(-100, 100) for _ in range(random.randint(5, 15))]
print('Expected:', max(values))
ll = LinkedList(values)
print(ll)
max_val = remove_max(ll)
print('Received:', max_val, end='\n\n')

while ll:
    print('Expected:', max(ll), '\tRemoved:', remove_max(ll))

Expected: 84
-10 -> 84 -> 7 -> 29 -> 40
Received: 84

Expected: 40 	Removed: 40
Expected: 29 	Removed: 29
Expected: 7 	Removed: 7
Expected: -10 	Removed: -10


### #4
#### Implementacja funkcji testującej, czy lista jest prawidłowo posortowana
##### (to nie jest częścią zadania)

In [7]:
import random

def test_sort(sort_fn, *, samples=20):
    passed = 0
    for i in range(samples):
        values = [random.randint(-100, 100) for _ in range(random.randint(5, 30))]
        expected = sorted(values)
        ll = LinkedList(values)
        sort_fn(ll)
        result = list(ll)
        passed += result == expected
        print(f'#{i+1} Loop (passed: {passed}/{samples}):')
        print(f'Proper answer: {expected}')
        print(f'After sorting: {result}')
        print()
    print(f'Total passed tests: {passed}/{samples}')
    print(f'A sorting algorithm is implemented {"correctly" if passed == samples else "improperly"}')

### #4 a) sortowanie przez wstawianie

In [8]:
def insert_node(ll, node):  # Inserts node in a right position maintaining the ascending order
    # Prepend the node
    if node.val < ll.head.val:
        node.next = ll.head
        ll.head = node
    # Insert the node before a greater one
    else:
        curr = ll.head
        while node.val > curr.next.val:
            curr = curr.next
        node.next = curr.next
        curr.next = node
    ll.length += 1
    
def insertion_sort(ll):
    if len(ll) < 2: return
    
    prev = ll.head
    while prev.next: # We start from the second node (prev.next)
        # If a current node (prev.next) has a value lower than a prev node, we have to
        # shift this node to a right position before.
        if prev.next.val < prev.val:
            removed = ll.remove_next_node(prev)       # We removed a curr node
            insert_node(ll, removed) # Now we insert this node in a right position
        # We can skip a current node otherwise.
        else:
            prev = prev.next

###### Przykładowy test

In [9]:
test_sort(insertion_sort, samples=100)

#1 Loop (passed: 1/100):
Proper answer: [-86, -84, -82, -77, -76, -73, -65, -51, -44, -40, -26, -11, 5, 9, 28, 65, 85, 89, 92, 95]
After sorting: [-86, -84, -82, -77, -76, -73, -65, -51, -44, -40, -26, -11, 5, 9, 28, 65, 85, 89, 92, 95]

#2 Loop (passed: 2/100):
Proper answer: [-66, -41, -39, 40, 50, 72, 85]
After sorting: [-66, -41, -39, 40, 50, 72, 85]

#3 Loop (passed: 3/100):
Proper answer: [-98, -90, -89, -85, -72, -65, -60, -48, -37, -26, -8, 0, 12, 14, 18, 19, 24, 29, 38, 50, 51, 70, 70, 75, 83, 94, 98]
After sorting: [-98, -90, -89, -85, -72, -65, -60, -48, -37, -26, -8, 0, 12, 14, 18, 19, 24, 29, 38, 50, 51, 70, 70, 75, 83, 94, 98]

#4 Loop (passed: 4/100):
Proper answer: [-98, -47, -22, 88, 91, 94]
After sorting: [-98, -47, -22, 88, 91, 94]

#5 Loop (passed: 5/100):
Proper answer: [-92, -70, -68, -66, -61, -60, -60, -58, -56, -46, -37, -27, -18, -14, -1, 9, 17, 19, 60, 61, 64, 65, 73, 74, 82, 91]
After sorting: [-92, -70, -68, -66, -61, -60, -60, -58, -56, -46, -37, -27, -18,

### #4 b) sortowanie przez wybieranie

In [10]:
def selection_sort(ll):
    if len(ll) < 2: return
    
    first = ll.head
    
    # Loop while a first pointer is not the last element
    while first.next:
        min_node = first
        second = first.next
        while second:
            # Look for a currently minimal value
            if second.val < min_node.val:
                min_node = second
            second = second.next
        # Swap values in order to make them sorted
        min_node.val, first.val = first.val, min_node.val
        first = first.next

###### Przykładowy test

In [11]:
test_sort(selection_sort, samples=100)

#1 Loop (passed: 1/100):
Proper answer: [-93, -92, -91, -80, -56, -55, -36, -25, -25, 2, 7, 13, 29, 31, 36, 44, 46, 48, 51, 89, 89, 91, 92]
After sorting: [-93, -92, -91, -80, -56, -55, -36, -25, -25, 2, 7, 13, 29, 31, 36, 44, 46, 48, 51, 89, 89, 91, 92]

#2 Loop (passed: 2/100):
Proper answer: [-94, -80, -77, -75, -62, -54, -47, -37, -28, -7, 9, 11, 11, 17, 22, 26, 32, 33, 33, 37, 43, 44, 45, 52, 71, 81, 83, 95, 95]
After sorting: [-94, -80, -77, -75, -62, -54, -47, -37, -28, -7, 9, 11, 11, 17, 22, 26, 32, 33, 33, 37, 43, 44, 45, 52, 71, 81, 83, 95, 95]

#3 Loop (passed: 3/100):
Proper answer: [-89, -77, -74, -73, -70, -68, -67, -58, -51, -50, -25, -17, -13, -9, -9, -6, 0, 15, 31, 40, 50, 52, 68, 84]
After sorting: [-89, -77, -74, -73, -70, -68, -67, -58, -51, -50, -25, -17, -13, -9, -9, -6, 0, 15, 31, 40, 50, 52, 68, 84]

#4 Loop (passed: 4/100):
Proper answer: [-93, -92, -83, -78, -75, -69, -61, -54, -46, -44, -39, -36, -33, -30, -6, -6, 1, 8, 29, 38, 45, 59, 74, 75, 80]
After sorti

Proper answer: [-92, -70, -1, -1, 3, 4, 7, 41, 53, 67, 80, 99]
After sorting: [-92, -70, -1, -1, 3, 4, 7, 41, 53, 67, 80, 99]

#69 Loop (passed: 69/100):
Proper answer: [-78, -70, -54, -53, -50, -46, -43, -42, -29, -12, -9, 8, 10, 10, 20, 22, 34, 35, 53, 62, 63, 75, 81, 82]
After sorting: [-78, -70, -54, -53, -50, -46, -43, -42, -29, -12, -9, 8, 10, 10, 20, 22, 34, 35, 53, 62, 63, 75, 81, 82]

#70 Loop (passed: 70/100):
Proper answer: [-88, -79, -78, -72, -64, -49, -42, -21, 5, 26, 66, 80, 87, 88, 88]
After sorting: [-88, -79, -78, -72, -64, -49, -42, -21, 5, 26, 66, 80, 87, 88, 88]

#71 Loop (passed: 71/100):
Proper answer: [-99, -65, -46, -10, -6, 8, 18, 87]
After sorting: [-99, -65, -46, -10, -6, 8, 18, 87]

#72 Loop (passed: 72/100):
Proper answer: [-86, -81, -74, -69, -65, -63, -55, -54, -52, -50, -46, -29, -29, -23, -21, -19, -17, 6, 7, 40, 44, 52, 60, 64, 67, 77, 87, 90, 99]
After sorting: [-86, -81, -74, -69, -65, -63, -55, -54, -52, -50, -46, -29, -29, -23, -21, -19, -17, 6, 7

### #4 (dodatkowo) c) sortowanie bąbelkowe

##### Poprzez zamianę wartości (działa tylko, gdy węzły przechowują pojedyncze wartości, np. liczby)    

In [12]:
def bubble_sort(ll):
    if len(ll) < 2: return
    
    marker = ll.head
    limit = None  # We use limit in order to speed up sorting (after each inner loop we are sure
                  # that on the position that we finished is placed the currently greatest value)
    while marker.next:
        curr = ll.head
        while curr.next is not limit:
            if curr.next.val < curr.val:
                curr.val, curr.next.val = curr.next.val, curr.val
            curr = curr.next
        limit = curr
#         print(limit.val)
        marker = marker.next

###### Przykładowy test

In [13]:
test_sort(bubble_sort, samples=100)

#1 Loop (passed: 1/100):
Proper answer: [-92, -88, -76, -56, -54, -49, -40, -24, 0, 1, 8, 12, 35, 70, 91, 96, 100]
After sorting: [-92, -88, -76, -56, -54, -49, -40, -24, 0, 1, 8, 12, 35, 70, 91, 96, 100]

#2 Loop (passed: 2/100):
Proper answer: [-97, -88, -86, -69, -69, -60, -58, -56, -47, -30, -17, -14, -13, -5, 2, 8, 13, 20, 42, 60, 65, 76, 85, 86, 87, 90, 90, 92, 98]
After sorting: [-97, -88, -86, -69, -69, -60, -58, -56, -47, -30, -17, -14, -13, -5, 2, 8, 13, 20, 42, 60, 65, 76, 85, 86, 87, 90, 90, 92, 98]

#3 Loop (passed: 3/100):
Proper answer: [-92, -44, -4, -1, 50, 83, 89]
After sorting: [-92, -44, -4, -1, 50, 83, 89]

#4 Loop (passed: 4/100):
Proper answer: [-77, -73, -70, -62, -60, -49, -33, -25, -19, -11, -2, 15, 15, 16, 18, 41, 44, 60, 61, 68, 68, 70, 89]
After sorting: [-77, -73, -70, -62, -60, -49, -33, -25, -19, -11, -2, 15, 15, 16, 18, 41, 44, 60, 61, 68, 68, 70, 89]

#5 Loop (passed: 5/100):
Proper answer: [-89, -89, -58, -57, -7, 6, 17, 46, 60, 72, 72, 94]
After sort

#94 Loop (passed: 94/100):
Proper answer: [-99, -96, -88, -68, -47, -40, -32, 54, 63, 65, 90]
After sorting: [-99, -96, -88, -68, -47, -40, -32, 54, 63, 65, 90]

#95 Loop (passed: 95/100):
Proper answer: [-96, -87, -81, -80, -68, -57, -53, -44, -43, -11, -10, -7, 4, 15, 36, 38, 47, 52, 54, 60, 71, 83, 94]
After sorting: [-96, -87, -81, -80, -68, -57, -53, -44, -43, -11, -10, -7, 4, 15, 36, 38, 47, 52, 54, 60, 71, 83, 94]

#96 Loop (passed: 96/100):
Proper answer: [-41, -33, -26, 43, 52, 85]
After sorting: [-41, -33, -26, 43, 52, 85]

#97 Loop (passed: 97/100):
Proper answer: [-62, -41, -36, -27, -24, -13, -9, -8, -8, -3, 6, 27, 38, 84, 100]
After sorting: [-62, -41, -36, -27, -24, -13, -9, -8, -8, -3, 6, 27, 38, 84, 100]

#98 Loop (passed: 98/100):
Proper answer: [-89, -88, -73, -52, -47, -42, -41, -29, -24, -20, -18, -13, -1, 36, 36, 68, 88, 98]
After sorting: [-89, -88, -73, -52, -47, -42, -41, -29, -24, -20, -18, -13, -1, 36, 36, 68, 88, 98]

#99 Loop (passed: 99/100):
Proper answer

##### Poprzez modyfikację wskaźników (bardziej ogólne - węzły mogą przechowywać dowolne dane - sortowanie według klucza (wartości) przestawia całe węzły, a nie zamienia danych, jak to powyżej)

In [14]:
def swap_nodes(first, second, third):
    second.next = third.next
    third.next = second
    first.next = third


def bubble_sort(ll):
    if len(ll) < 2: return
    # Store a loops count of an outer loop and a limit node
    loops = len(ll) - 1
    limit = None
    # Add a sentinel node
    ll.prepend(None)
    # Loop over a Linked List and modify pointers (if necessary) in order to swap nodes
    for _ in range(loops):
        prev = ll.head
        while prev.next.next is not limit:
            if prev.next.next.val < prev.next.val:
                swap_nodes(prev, prev.next, prev.next.next)
            prev = prev.next
        limit = prev.next
#         print(limit.val)
    # Remove a sentinel node
    ll.popleft()

###### Przykładowy test

In [15]:
test_sort(bubble_sort, samples=100)

#1 Loop (passed: 1/100):
Proper answer: [-80, -68, -44, -11, -11, 16, 25, 27, 40, 49, 56, 69]
After sorting: [-80, -68, -44, -11, -11, 16, 25, 27, 40, 49, 56, 69]

#2 Loop (passed: 2/100):
Proper answer: [-81, -77, -64, -60, -21, 14, 31, 49, 56]
After sorting: [-81, -77, -64, -60, -21, 14, 31, 49, 56]

#3 Loop (passed: 3/100):
Proper answer: [-98, -95, -89, -88, -83, -68, -63, -59, -29, -23, -2, 9, 15, 29, 48, 49, 54, 55, 57, 60, 60, 67, 69, 69, 82, 86, 95]
After sorting: [-98, -95, -89, -88, -83, -68, -63, -59, -29, -23, -2, 9, 15, 29, 48, 49, 54, 55, 57, 60, 60, 67, 69, 69, 82, 86, 95]

#4 Loop (passed: 4/100):
Proper answer: [-89, -85, -79, -77, -75, -67, -59, -53, -51, -37, -33, -20, -15, -12, -2, -1, 10, 14, 14, 32, 38, 39, 48, 50, 53, 67, 68, 73, 78, 100]
After sorting: [-89, -85, -79, -77, -75, -67, -59, -53, -51, -37, -33, -20, -15, -12, -2, -1, 10, 14, 14, 32, 38, 39, 48, 50, 53, 67, 68, 73, 78, 100]

#5 Loop (passed: 5/100):
Proper answer: [-71, -55, -51, -44, -23, -3, 8, 23,

#41 Loop (passed: 41/100):
Proper answer: [-96, -82, -78, -72, -56, -50, -49, -24, 5, 36, 42, 47, 58, 68, 73, 89, 89]
After sorting: [-96, -82, -78, -72, -56, -50, -49, -24, 5, 36, 42, 47, 58, 68, 73, 89, 89]

#42 Loop (passed: 42/100):
Proper answer: [-89, -68, -46, -37, -23, -19, -5, 5, 8, 12, 13, 32, 35, 38, 45, 48, 55, 58, 85, 88, 99, 100, 100]
After sorting: [-89, -68, -46, -37, -23, -19, -5, 5, 8, 12, 13, 32, 35, 38, 45, 48, 55, 58, 85, 88, 99, 100, 100]

#43 Loop (passed: 43/100):
Proper answer: [-94, -91, -85, -75, -47, -7, -3, 10, 13, 15, 22, 22, 23, 24, 35, 37, 41, 66, 78, 91, 97]
After sorting: [-94, -91, -85, -75, -47, -7, -3, 10, 13, 15, 22, 22, 23, 24, 35, 37, 41, 66, 78, 91, 97]

#44 Loop (passed: 44/100):
Proper answer: [-94, -85, -66, -54, -48, -24, -3, 30, 42, 66, 75]
After sorting: [-94, -85, -66, -54, -48, -24, -3, 30, 42, 66, 75]

#45 Loop (passed: 45/100):
Proper answer: [-99, -99, -78, -74, -62, -44, -38, -24, 4, 5, 39, 44, 71, 92]
After sorting: [-99, -99, -78, 